In [2]:
import pandas as pd

In [13]:
products_df = pd.read_csv(r'./facemasks/products.tsv', sep='\t')
products_df.head()

,product_id,product_name,product_price,price_currency,product_availability,product_url,source_url
0,103205,"Hwipure, Disposable KF94 ( N95 / KN95/ FFP2 ) ...",2.95,AUD,http://schema.org/InStock,https://au.iherb.com/pr/Hwipure-Disposable-KF9...,https://au.iherb.com/c/protective-face-masks
1,101774,"HIGUARD, Disposable KF94 ( N95 / KN95/ FFP2 ) ...",2.95,AUD,http://schema.org/InStock,https://au.iherb.com/pr/HIGUARD-Disposable-KF9...,https://au.iherb.com/c/protective-face-masks
2,101955,"SunJoy, KN95, Professional Protective Disposab...",8.86,AUD,http://schema.org/InStock,https://au.iherb.com/pr/SunJoy-KN95-Profession...,https://au.iherb.com/c/protective-face-masks
3,103838,"Lozperi, Copper Mask, Adult, Black, 1 Mask",6.85,AUD,http://schema.org/InStock,https://au.iherb.com/pr/Lozperi-Copper-Mask-Ad...,https://au.iherb.com/c/protective-face-masks
4,102734,"Zidian, Disposable Protective Mask, 50 Pack",15.35,AUD,http://schema.org/InStock,https://au.iherb.com/pr/Zidian-Disposable-Prot...,https://au.iherb.com/c/protective-face-masks


In [14]:
# product_id is irrelevant to our analysis
# currency is AUD for all products, so it does not add information 
# all the products are in stock
# source_url just leads to a page with search result for 'protective face masks'

products_df.drop(['product_id', 'price_currency', 'product_availability', 'source_url'], axis=1, inplace=True)
products_df.head()

,product_name,product_price,product_url
0,"Hwipure, Disposable KF94 ( N95 / KN95/ FFP2 ) ...",2.95,https://au.iherb.com/pr/Hwipure-Disposable-KF9...
1,"HIGUARD, Disposable KF94 ( N95 / KN95/ FFP2 ) ...",2.95,https://au.iherb.com/pr/HIGUARD-Disposable-KF9...
2,"SunJoy, KN95, Professional Protective Disposab...",8.86,https://au.iherb.com/pr/SunJoy-KN95-Profession...
3,"Lozperi, Copper Mask, Adult, Black, 1 Mask",6.85,https://au.iherb.com/pr/Lozperi-Copper-Mask-Ad...
4,"Zidian, Disposable Protective Mask, 50 Pack",15.35,https://au.iherb.com/pr/Zidian-Disposable-Prot...


In [26]:
reviews_df = pd.read_csv(r'./facemasks/reviews.tsv', sep='\t')
reviews_df.head()

,abuseCount,customerNickname,helpfulNo,helpfulYes,id,imagesCount,languageCode,postedDate,productId,profileInfo.ugcSummary.answerCount,profileInfo.ugcSummary.reviewCount,ratingValue,reviewText,reviewTitle,reviewed,score,languageCode.1,translation.reviewText,translation.reviewTitle
0,0,iHerb Customer,0,6,05c2b17e-c28d-4792-930d-27e787d8d4ad,1,en-US,2021-01-27T09:04:10.569Z,103839,0.0,34.0,50,The mask quality and the color is good. It fit...,Dotted Pattern Is Nice,True,1.614071e+09,en-US,NaN,NaN
1,0,iHerb Customer,0,0,80e44af8-2edf-4b81-a80a-7e7888d03cc0,0,ru-RU,2021-02-07T00:56:39.055Z,103839,0.0,37.0,50,Внуку очень понравилось. Удобная маска.,Прекрасно!,False,1.612659e+09,ru-RU,The grandson really liked it. Comfortable mask.,en-US
2,0,iHerb Customer,0,0,9a76e047-21e4-4da3-8b50-9d2396519b6b,0,en-US,2021-02-06T21:40:02.886Z,103839,0.0,3.0,40,Easy to put on & comfortable to wear.,Good,False,1.612648e+09,en-US,NaN,NaN
3,0,InnaIgorevna,0,0,2890ac54-8707-418e-be3e-8d46231e3672,0,ru-RU,2021-02-05T16:29:28.906Z,103839,0.0,16.0,40,"Тонкая, приятная к телу маска. Расцветка тоже ...",Хорошо,False,1.612543e+09,ru-RU,A thin mask that is pleasant to the body. I li...,en-US
4,0,iHerb Customer,0,0,9db33354-0457-4efa-bc9c-b5f7ee0eff31,0,ru-RU,2021-02-05T09:43:42.367Z,103839,0.0,19.0,50,Отличная маска! Мне подошла идеально. Слегка е...,Прекрасно!,False,1.612518e+09,ru-RU,Great mask! It suited me perfectly. There is a...,en-US


In [29]:
for name in reviews_df.columns:
    print(name, eval(f'reviews_df[\'{name}\'].unique()'))

abuseCount [0 1 2 3 4]
customerNickname ['iHerb Customer' 'InnaIgorevna' 'Maxchen' 'Mahaab' 'Michalgw' 'Tallula'
 'KatrinSolntce' 'Erez Opincaru' 'NanoxBeauty' 'KavynovskayY'
 'MichalEliad' 'Kseniasitcher' 'Eden32' 'Jennylyub' 'djagi' 'Ilove Health'
 'iHerb Client' 'GoldenElephant' 'omilla' 'ElenaV' 'Mhholding' 'MisterKot'
 'AstroNut' 'AlekseyMoscow' 'Read more reviews' 'TMC' 'INNAg' 'emmaiherb'
 'teymiah' 'Cris' 'Bkr-banamah' 'Vergissmeinnicht' 'SuperMOM' 'Sputnik'
 'iherb2reviews' 'iherbie' 'Alico4ka90' 'Innaaa' 'Guzel' 'YuliyaMSM'
 'Lyudmila' 'OlaSpas' 'HealthyHealthy' 'Alexooo' 'Jomana' 'Mariam12'
 'Vitarka' 'Melani' 'LoCarb4ever' 'HfromSPhilly' 'GGhappy' 'mamamiii'
 'Helga13' 'Rihamuae' 'NataliCat' 'aryeesh' 'Warda30' 'Seja10'
 'afinaastenis' 'AlinaGromov' 'REPLYzzz' 'Margrev' 'Salmiakkinen'
 'Karina91' 'Maryam22' 'AaronA' 'iAsrar' 'AOE YY 0137' 'Andrew' 'Lisaveta'
 'AalenaS' 'Adamysh' 'Elvira C' 'Fanherb' 'JannetZ' 'MarinaK5' 'AnnVladi'
 'Rina-Irina' 'Maya' 'JFJane' 'Oksana Novik

In [24]:
reviews_df.drop(['customerNickname', 'id', 'productId'], axis=1, inplace=True)

Index(['abuseCount', 'customerNickname', 'helpfulNo', 'helpfulYes', 'id',
       'imagesCount', 'languageCode', 'postedDate', 'productId',
       'profileInfo.ugcSummary.answerCount',
       'profileInfo.ugcSummary.reviewCount', 'ratingValue', 'reviewText',
       'reviewTitle', 'reviewed', 'score', 'languageCode.1',
       'translation.reviewText', 'translation.reviewTitle'],
      dtype='object')